# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

Признаки
* DateCrawled — дата скачивания анкеты из базы
* VehicleType — тип автомобильного кузова
* RegistrationYear — год регистрации автомобиля
* Gearbox — тип коробки передач
* Power — мощность (л. с.)
* Model — модель автомобиля
* Kilometer — пробег (км)
* RegistrationMonth — месяц регистрации автомобиля
* FuelType — тип топлива
* Brand — марка автомобиля
* Repaired — была машина в ремонте или нет
* DateCreated — дата создания анкеты
* NumberOfPictures — количество фотографий автомобиля
* PostalCode — почтовый индекс владельца анкеты (пользователя)
* LastSeen — дата последней активности пользователя


Целевой признак
* Price — цена (евро)

## Подготовка данных

In [ ]:
!pip install phik -q
!pip install --upgrade scikit-learn -q
!pip install lightgbm --upgrade -q

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from time import time

import phik
from phik import phik_matrix
from phik.report import plot_correlation_matrix

from sklearn.model_selection import (
    train_test_split,
    cross_val_score,
    RandomizedSearchCV
)

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

from sklearn.preprocessing import (
    OneHotEncoder,
    OrdinalEncoder, 
    StandardScaler, 
    MinMaxScaler,
    RobustScaler,
)
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor

from sklearn.metrics import mean_squared_error

from sklearn.dummy import DummyRegressor


RANDOM_STATE = 42
sns.set_style('darkgrid')
threshold = 10
top_n = 10

In [ ]:
df1 = 'C:/Users/Gpets/Data With Python/datasets/datasets/autos.csv'

df_alt = '/datasets/autos.csv'

def load_data(primary_path, secondary_path):
    if os.path.exists(primary_path):
        try:
            return pd.read_csv(primary_path, sep=',')
        except:
            return pd.read_csv(primary_path, sep=';')
    elif os.path.exists(secondary_path):
        try:
            return pd.read_csv(secondary_path, sep=',')
        except:
            return pd.read_csv(secondary_path, sep=';')
    else:
        print(f"Файл не найден: {primary_path} или {secondary_path}")
        return None


df = load_data(df1, df_alt)

In [ ]:
df.head()

### Приведение названия столбцов к snake_case

In [ ]:
frames = [
    df
]

for i in frames:
    i.columns = [re.sub(r'(?<!^)(?=[A-Z])', '_', x).replace(' ', '_').lower() for x in i.columns]

In [ ]:
df.columns

Удалим неинформативные признаки для определения стоимости автомобиля. Это:

* `date_crawled` - дата скачивания анкеты из базы

* `date_created` - дата создания анкеты

* `number_of_pictures` - количество фотографий автомобиля

* `postal_code` - почтовый индекс владельца анкеты (пользователя)

* `last_seen` - дата последней активности пользователя

Теперь выводим основную информацию о датафрейме с помощью метода `describe()` и `info()`

In [ ]:
df.info()

У нас 354.369 записей пользователей, есть пропущенные значения в столбцах, типы данных в целом, соответствуют ожиданиям.

Посмотрим подробнее на пропуски в таблице

In [ ]:
df.isnull().sum()

В данных имеются пропуски в 5 признаках: `vehicle_type`, `gearbox`, `model`, `fuel_type` и `repaired`

Заменить их на медианы или средние значения нежелателоьно, так как каждая запись индивидуальна, и замена пропусков таким способом может сильно исказить нашу модель. Заполним пропуски значением `unknown`

In [ ]:
columns_to_fill = ['vehicle_type', 'gearbox', 'model', 'fuel_type', 'repaired']
df[columns_to_fill] = df[columns_to_fill].fillna('unknown')

In [ ]:
df.isnull().sum()

Теперь проведем проверку на явные дубликаты

In [ ]:
print('Кол-во строк и столбцов в датасете:',df.shape)
print("Число анкет:", df.index.nunique())
print('Число дубликатов',df.duplicated().sum())

Нашлось немного дубликатов, избавимся от них

In [ ]:
df = df.drop_duplicates()
print('Число дубликатов',df.duplicated().sum())

Явных дубликатов не наблюдается, теперь проверим на наличие неявных

Создадим цикл который пройдется по категориальным признакам и покажет в них уникальные значения

In [ ]:
categorical_columns = ['vehicle_type', 'gearbox', 'model', 'fuel_type', 'brand', 'repaired']

for column in categorical_columns:
    print(f'Проверка столбца: {column}')
    # Приводим все значения к нижнему регистру и удаляем пробелы
    cleaned_values = df[column].str.strip().str.lower()
    # Выводим уникальные значения и их частоту
    print(cleaned_values.value_counts())
    print()


Неявных дубликатов нет

In [ ]:
df.describe()

У нас много аномалий в данных, например:

* Цена автомобиля не может быть равна 0

* Год регистрации не может быть равным 1000 или 9999 

* Мощность машины не может быть равна 0 или 20,000 л.c

* Месяц регистрации не может быть равен 0 

Надо провести корректировку признаков

In [ ]:
# Обработка аномалий в столбце Price
df = df[df['price'] > 1000]  # Заменяем 0

In [ ]:
# Обработка аномалий в столбце Power
df = df[(df['power'] > 80) & (df['power'] <= 1500)]

Определим год выгрузки по максимальной дате создания анкеты

In [ ]:
df = df.assign(last_seen=pd.to_datetime(df['date_created'], errors='coerce'))
df['date_created'].max()

In [ ]:
current_year = 2016
df = df[(df['registration_year'] >= 1970) & (df['registration_year'] <= current_year)]

In [ ]:
df = df[df['registration_month'] > 0]
# Проверим, как изменения отразились на данных
df.describe()

Рассмотрим каждый столбец:

* **`Цена (price):`**

    * Средняя цена составляет 6621.08, с диапазоном от 1001 до 20000. 
    
    * Разброс значений высокий, что указывает на наличие как дешевых, так и дорогих автомобилей.
    

* **`Год регистрации (registration_year):`**

    * Средний год регистрации — 2004.61, с минимальным значением 1970 и максимальным — 2020.
    
    * Преобладание автомобилей, зарегистрированных в 2000-х годах (период от 2001 до 2008).
    

* **`Мощность (power):`**

    * Средняя мощность автомобилей — 144.83 л.с., с минимальным значением 81 л.с. и максимальным — 1500 л.с.
    
    * Есть автомобили с очень высокой мощностью, что может указывать на присутствие спортивных или мощных моделей.
    

* **`Пробег (kilometer):`**

    * Средний пробег автомобилей — 128,127 км, с минимальным значением 5000 км и максимальным — 150,000 км.
    
    * Большинство автомобилей имеют пробег в пределах 125,000–150,000 км.
    

* **`Месяц регистрации (registration_month):`**

    * Средний месяц регистрации — 6 (июнь), что указывает на равномерное распределение автомобилей по месяцам.
    
Данные содержат автомобили с разнообразными характеристиками, включая модели с различным пробегом и мощностью, что предполагает широкий спектр в рассматриваемой выборке. Разброс в ценах и мощности также подтверждает наличие как более дешевых, так и более дорогих автомобилей.

### Исследовательский анализ данных

Приступим к анализу каждого признака. Для начала разделим наши данные на колличественные и категориальные признаки

In [ ]:
df.columns

Расмотрим только ключевые признаки для модели

In [ ]:
quantitative_features = ['price', 'power', 'kilometer']


categorical_features = ['vehicle_type', 'gearbox', 'model','fuel_type',
                        'brand', 'repaired','registration_year']

#### Колличественные признаки

Построим графики для анализа количественных признаков

In [ ]:
plt.figure(figsize=(14, 12))
for i, feature in enumerate(quantitative_features, 1):
    plt.subplot(4, 2, i)
    sns.histplot(data=df, x=feature, kde=True)
    plt.title(f'Распределение {feature}')
plt.tight_layout()
plt.show()

* **`Цена (price):`** Распределение цен имеет явный правый скошенный характер, с большей концентрацией значений в нижней части диапазона. Это говорит о том, что большинство автомобилей имеют низкую цену, а меньшая часть попадает в более дорогие сегменты.


* **`Мощность (power):`** Распределение мощности также скошено вправо. Основная масса автомобилей имеет мощность около 100–150 л.с., но встречаются и модели с гораздо большей мощностью, что подтверждается присутствием длинного хвоста в правой части распределения.


* **`Пробег (kilometer):`** Распределение пробега похожее на нормальное с небольшим скошением вправо. Большинство автомобилей имеют пробег в районе 125,000–150,000 км, что подтверждается пиком в этой области. Однако встречаются автомобили с гораздо меньшим пробегом (первичный рынок), а также с очень высоким пробегом.

Теперь взглянем на наши данные с помощью боксплота

In [ ]:
for feature in quantitative_features:
    plt.figure(figsize=(10, 6))
    sns.boxplot(data=df, y=feature)
    plt.title(f'Boxplot для {feature}')
    plt.show()

* **`Цена (price):`** Boxplot показывает значительное количество выбросов с высокими ценами, что подтверждает, что в данных присутствуют дорогие автомобили. Медиана лежит ближе к нижнему диапазону, а большинство автомобилей находятся в ценовом сегменте ниже средней стоимости.


* **`Мощность (power):`** Мощность также имеет значительные выбросы с большими значениями (выше 300 л.с.), что соответствует присутствию мощных автомобилей. Медиана мощности находится около 130–150 л.с., что также подтверждается распределением.


* **`Пробег (kilometer):`** Для пробега также имеются выбросы, что подтверждает наличие автомобилей с очень низким пробегом (например, новые автомобили) и очень высокими значениями (автомобили с долгим сроком эксплуатации). Медиана расположена ближе к верхнему пределу диапазона.

#### Категориальные признаки

In [ ]:
for feature in categorical_features:
    plt.figure(figsize=(8, 6))
    
    if df[feature].nunique() > threshold:
        top_categories = df[feature].value_counts().head(top_n)
        sns.barplot(y=top_categories.index, x=top_categories.values, palette='colorblind')
        plt.xlabel('Количество')
        plt.ylabel(feature.replace('_', ' ').capitalize())
        plt.title(f'Топ-{top_n} значений по {feature.replace("_", " ").capitalize()}')
    else:
        sns.countplot(y=feature, data=df, palette='colorblind')
        plt.xlabel('Количество')
        plt.ylabel(feature.replace('_', ' ').capitalize())
        plt.title(f'Распределение по {feature.replace("_", " ").capitalize()}')
    
    plt.show()

Краткий вывод по категориальным данным:

* **`Типы автомобилей (vehicle_type):`** Наибольшее количество автомобилей — это седаны (54,819), за ними следуют универсалы (41,482) и автобусы (19,561). Меньше всего встречаются автомобили категории "другое" (1,184) и "неизвестно" (7,808). Это показывает, что более популярные типы кузовов, такие как седаны и универсалы, составляют основную массу рынка.


* **`Тип коробки передач (gearbox):`** Большинство автомобилей имеют механическую коробку передач (124,977), в то время как автомобили с автоматической коробкой передач встречаются реже (44,772). Количество автомобилей с неизвестным типом коробки передач также незначительно (1,668).


* **`Месяц регистрации (registration_month):`** Наибольшее количество автомобилей зарегистрировано в сентябре (18,195), а наименьшее — в январе (12,144). Распределение месяцев регистрации достаточно равномерное, с небольшим пиком в летние месяцы, что может указывать на сезонность регистрации автомобилей.


* **`Тип топлива (fuel_type):`** Наиболее популярное топливо — это бензин (91,244), за ним следует газ (69,180). Примечание: существует небольшой процент автомобилей с альтернативными источниками энергии, таких как гибриды (142) и электрические автомобили (17), что подтверждает низкую популярность экологичных технологий в рассматриваемых данных.


* **`Марки автомобилей (brand):`** Наиболее популярные марки — Volkswagen, BMW, Mercedes-Benz и Audi. Эти бренды значительно превосходят по количеству другие марки. Меньше всего представлены автомобили марок, таких как Lada, Daewoo и Dacia, что подтверждает небольшое присутствие этих брендов на рынке.


* **`Ремонт (repaired):`** Большинство автомобилей не ремонтировались (142,153), а небольшая часть была отремонтирована (10,873). Также имеется категория с неизвестным статусом ремонта (18,391), что может быть связано с отсутствием информации о состоянии автомобиля.


* **`Год регистрации (registration_year):`** Наибольшее количество автомобилей зарегистрировано в 2006 году (13,709), за ним следуют 2005 и 2004 годы. Это указывает на значительное количество автомобилей, произведенных в начале 2000-х. Количество автомобилей с годами регистрации 2020 и ранее постепенно снижается, что подтверждает тенденцию к старению автопарка.

Теперь для нашей модели удалим неинформативные признаки из данных, для лучшей работы модели

Это:

* `date_crawled` - дата скачивания анкеты из базы

* `date_created` - дата создания анкеты

* `number_of_pictures` - количество фотографий автомобиля

* `postal_code` - почтовый индекс владельца анкеты (пользователя)

* `last_seen` - дата последней активности пользователя

* `registration_month` - месяц регистрации автомобиля

In [ ]:
df = df.drop(['date_crawled', 'date_created', 'number_of_pictures', 'postal_code', 'last_seen', 'registration_month'], axis=1)

Проверим наши данные на наличие дубликатов после удаления ненужных признаков

In [ ]:
print('Кол-во строк и столбцов в датасете:',df.shape)
print("Число анкет:", df.index.nunique())
print('Число дубликатов',df.duplicated().sum())

Удалим дубликаты

In [ ]:
df = df.drop_duplicates()
print('Число дубликатов',df.duplicated().sum())

### Корреляционный анализ

Построим корреляционную матрицу с помощью phik и сделаем некоторые выводы

In [ ]:
# Пересчитываем Phi_K корреляционную матрицу без 'id'
corr_matrix = df.phik_matrix(interval_cols=['price', 'power'])
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.title('Phi_K Корреляционная матрица')
plt.show()

В корреляционной матрице видно, что `model` имеет очень высокую корреляцию с `vehicle_type` (0.902) и `brand` (1). Это указывает на то, что переменная model сильно зависит от этих переменных и может быть избыточной.

In [ ]:
df = df.drop(columns=['model'])
df.head()

In [ ]:
corr_matrix = df.phik_matrix(interval_cols=['price', 'power'])
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.title('Phi_K Корреляционная матрица')
plt.show()

Из корреляционной матрицы можно сделать следующие выводы:

* Высокая корреляция между `registration_year` и `vehicle_type` (0.708): Это говорит о том, что старые автомобили, скорее всего, имеют определенные типы кузовов, например, седаны или универсалы. Такое соотношение важно учитывать, так как более старые автомобили могут быть связаны с другими характеристиками.


* `price` имеет сильную корреляцию с `registration_year` (0.654): Цена автомобиля положительно коррелирует с годом регистрации, что подтверждает ожидание, что более новые автомобили стоят дороже.


* Корреляция между `price` и `power` (0.287): Это умеренная положительная корреляция, что указывает на то, что автомобили с большей мощностью, как правило, стоят дороже, хотя эта связь не является слишком сильной.


* Корреляция между `kilometer` и `registration_year` (0.642): Старые автомобили, как правило, имеют больший пробег, что подтверждается этим соотношением.


* `brand` и `vehicle_type` имеют достаточно сильную корреляцию (0.605): Определенные бренды чаще производят автомобили определенных типов кузова, что может быть полезным при анализе предпочтений потребителей.


* Низкая корреляция с `registration_month`, `fuel_type`, и `repaired`: Эти переменные имеют слабую или практически отсутствующую корреляцию с другими признаками, что может указывать на их независимость в контексте данного набора данных.


В целом, корреляционная матрица показывает, что некоторые переменные, такие как `registration_year`, `price`, и `vehicle_type`, оказывают значительное влияние на другие, и могут быть важными при построении модели.

### Вывод по подготовке данных

После предобработки данных мы очистили датасеты от неявных дубликатов и привели названия столбцов к единому формату. Преобразовали строки с числовыми значениями в нужные числовые типы. Все данные заполнены корректно, пропущенных значений нет. Мы также убедились, что распределение данных адекватное для последующего анализа. Датасет теперь готов для дальнейшего анализа и построения моделей.

## Обучение моделей

Подготовим выборки для обучения моделей.

In [ ]:
X = df.drop(columns=['price'])
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=RANDOM_STATE)

Сначала обозначим, какие данные требуют масштабирования, а какие — кодирования.

Введём обозначения для типов исходных данных

После предобработки данных мы очистили датасеты от неявных дубликатов и привели названия столбцов к единому формату. Преобразовали строки с числовыми значениями в нужные числовые типы. Все данные заполнены корректно, пропущенных значений нет. Мы также убедились, что распределение данных адекватное для последующего анализа. Датасет теперь готов для дальнейшего анализа и построения моделей.

In [ ]:
cols_ordinal = ['gearbox', 'vehicle_type', 'fuel_type', 'brand', 'repaired']
cols_to_scale = [ 'power', 'kilometer']

### Пайплайн preprocessor

Пайплайн для подготовки бинарных признаков.

In [ ]:
ordered_pipeline = Pipeline(
    [
        ('simpleImputer_before_ord',
         SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
        ('ordered',  OrdinalEncoder(
            handle_unknown='use_encoded_value', unknown_value=np.nan)),
        ('simpleImputer_after_ord',
         SimpleImputer(missing_values=np.nan, strategy='most_frequent'))
    ]
)
print(ordered_pipeline)

Создаем общий пайплайн подготовки данных

In [ ]:
data_preprocessor = ColumnTransformer(
    [
        ('ordered', ordered_pipeline, cols_ordinal),
        ('num', StandardScaler(), cols_to_scale),
    ], 
    remainder =  'passthrough'
)
print(data_preprocessor)

### Обучение моделей

Создадим пайплайн , который использует `ColumnTransformer` для обработки данных. Это позволяет обрабатывать количественные и категориальные признаки отдельно

In [ ]:
pipe_final = Pipeline([
    ('preprocessor', data_preprocessor),
    ('models', LinearRegression())
])

Мы обучим 2 модели: RandomForestRegressor и LGBMRegressor. Для каждой модели мы определим набор гиперпараметров, которые хотели бы оптимизировать с помощью RandomizedSearchCV.

In [ ]:
param_grid_rf = [
    {
        'models': [RandomForestRegressor(random_state=RANDOM_STATE)],
        'models__n_estimators': [10, 50, 100],
        'models__max_depth': [10, 20, None],
        'models__min_samples_split': [2, 5, 10],
        'models__min_samples_leaf': [1, 2, 4],
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), RobustScaler(), 'passthrough']
    }
]

In [ ]:
param_grid_lgbm =[
        {
        'models': [LGBMRegressor(random_state=RANDOM_STATE)],
        'models__n_estimators': [10, 50, 100],
        'models__max_depth': range(1, 11),
        'models__learning_rate': [0.0001, 0.001, 0.01, 0.1, 1.0],
        'models__boosting_type': ['gbdt', 'dart', 'goss'],
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), RobustScaler(), 'passthrough']
    }
]

Подбор гиперпараметров с использованием `RandomizedSearchCV`

### RandomForestRegressor

In [ ]:
randomized_search_rf = RandomizedSearchCV(
    pipe_final, 
    param_grid_rf, 
    cv=3,
    scoring='neg_mean_squared_error',
    random_state=RANDOM_STATE,
    n_jobs=-1
)

randomized_search_rf.fit(X_train, y_train)


best_model_rf = randomized_search_rf.best_estimator_
best_score_rf = randomized_search_rf.best_score_
rmse_rf = np.sqrt(-best_score_rf)

start_rf_fit = time()
best_model_rf.fit(X_train, y_train)
end_rf_fit = time()
rf_best_train_time = (end_rf_fit - start_rf_fit) / 60


start_rf_pred = time()
rf_preds = best_model_rf.predict(X_train)
end_rf_pred = time()
rf_pred_time = (end_rf_pred - start_rf_pred) / 60 

print('RandomizedSearchCV_RF - параметры:\n\n', best_model_rf)
print('RandomizedSearchCV_RF - Метрика лучшей модели на кросс-валидации:', rmse_rf)
print('RandomizedSearchCV_RF - Скорость обучения (мин):', rf_best_train_time)
print('RandomizedSearchCV_RF - Скорость предсказания (мин):', rf_pred_time)

### LGBMRegressor

In [ ]:
start_lgbm = time()
randomized_search_lgbm = RandomizedSearchCV(
    pipe_final, 
    param_grid_lgbm, 
    cv=5,
    scoring='neg_mean_squared_error',
    random_state=RANDOM_STATE,
    n_jobs=-1
)

randomized_search_lgbm.fit(X_train, y_train)


best_model_lgbm = randomized_search_lgbm.best_estimator_
best_score_lgbm = randomized_search_lgbm.best_score_
rmse_lgbm = np.sqrt(-best_score_lgbm)

start_lgbm_fit = time()
best_model_lgbm.fit(X_train, y_train)
end_lgbm_fit = time()
lgbm_best_train_time = (end_lgbm_fit - start_lgbm_fit) / 60

start_lgbm_pred = time()
lgbm_preds = best_model_lgbm.predict(X_train)
end_lgbm_pred = time()
lgbm_pred_time = (end_lgbm_pred - start_lgbm_pred) / 60 


print('RandomizedSearchCV_LGBM - параметры:\n\n', best_model_lgbm)
print('RandomizedSearchCV_LGBM - Метрика лучшей модели на кросс-валидации:', rmse_lgbm)
print('RandomizedSearchCV_LGBM - Скорость обучения (мин):', lgbm_best_train_time)
print('RandomizedSearchCV_LGBM - Скорость предсказания (мин):', lgbm_pred_time)

## Анализ моделей

Давайте взглянем на наши модели:

RandomForestRegressor:

* RMSE: 1928.5807779780596


* Скорость обучения (мин): 0.12312793334325155


* Скорость предсказания (мин): 0.01782900889714559

LGBMRegressor:
* RMSE: 1941.2658483176444


* Скорость обучения (мин): 0.04471121231714884


* Скорость предсказания (мин): 0.008946176369984944


Random Forest показывает немного лучшее RMSE, чем LGBM (1928.58 против 1941.27). Разница минимальна.

LGBM значительно быстрее обучается (3 секунды против 7 секунд у Random Forest).

LGBM быстрее делает предсказания (менее 1 секунды против 1 секунды у Random Forest).

Лучшая модель по всем признакам это `LGBMRegressor`

Проверим работу модели на тестовых данных

In [ ]:
y_pred_lgbm = best_model_lgbm.predict(X_test)

lgbm_mse = mean_squared_error(y_test, y_pred_lgbm)

lgbm_rmse = np.sqrt(lgbm_mse)

print("LightGBM RMSE:", lgbm_rmse)

Значение метрики RMSE меньше 2500. Все отлично

### Проверка на адекватность

Проверим модель на адекватность с помощью `DummyRegressor`

In [ ]:
# Создаем DummyRegressor
dummy_regressor = DummyRegressor(strategy="mean")

# Обучаем DummyRegressor на тренировочных данных
dummy_regressor.fit(X_train, y_train)

# Предсказания DummyRegressor
y_dummy_pred = dummy_regressor.predict(X_test)

# Вычисление RMSE для DummyRegressor вручную
dummy_mse = mean_squared_error(y_test, y_dummy_pred)
dummy_rmse = np.sqrt(dummy_mse)

print("DummyRegressor RMSE:", dummy_rmse)

if lgbm_rmse < dummy_rmse:
    print("LightGBM лучше DummyRegressor")
else:
    print("LightGBM хуже или равен DummyRegressor")

Проверку на адекватность пройдена.

## Вывод

Tребования заказчика включают:

* Скорость обучения

* Качество предсказаний

* Скорость предсказаний

В ходе работы были построены и проанализированы две модели: `RandomForestRegressor` и `LGBMRegressor.` По результатам тестирования и проверки на адекватность можно сделать следующие выводы:


* Качество предсказания:

    * RMSE модели LGBMRegressor на тестовых данных составило 1928.11, что меньше допустимого порога в 2500, установленного заказчиком.
    
    * Модель показала лучшие результаты по сравнению с DummyRegressor (RMSE: 4816.16) и успешно прошла проверку на адекватность.
    
    
* Скорость обучения и предсказания:

    * LGBMRegressor значительно быстрее обучается (3 секунды) и предсказывает (менее 1 секунды) по сравнению с RandomForestRegressor.
    
    
* Сравнение моделей:

    * Несмотря на небольшое превосходство RandomForestRegressor в метрике RMSE на этапе кросс-валидации, LGBMRegressor была выбрана как лучшая модель по всем критериям заказчика: качеству предсказания, скорости обучения, и времени предсказания.
    
    
Наша модель LGBMRegressor полностью удовлетворяет требованиям заказчика.